## Imports

In [1]:
# !pip install albumentations
# !pip install kaggle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# from google.colab import files
# files.upload()

In [4]:
!mkdir -p ~/.kaggle

!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [5]:
cd /content/drive/MyDrive/Colab Notebooks/homework/CIS5810/Final_Project NST

/content/drive/MyDrive/Colab Notebooks/homework/CIS5810/Final_Project NST


In [6]:
# !kaggle datasets download -d suyashdamle/cyclegan

In [16]:
!unzip cyclegan.zip -d "cycleGANdata"

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: cycleGANdata/vangogh2photo/vangogh2photo/trainB/2015-02-12 11:31:32.jpg  
  inflating: cycleGANdata/vangogh2photo/vangogh2photo/trainB/2015-02-12 12:30:32.jpg  
  inflating: cycleGANdata/vangogh2photo/vangogh2photo/trainB/2015-02-12 19:51:27.jpg  
  inflating: cycleGANdata/vangogh2photo/vangogh2photo/trainB/2015-02-12 20:53:44.jpg  
  inflating: cycleGANdata/vangogh2photo/vangogh2photo/trainB/2015-02-12 21:31:48.jpg  
  inflating: cycleGANdata/vangogh2photo/vangogh2photo/trainB/2015-02-13 09:22:26.jpg  
  inflating: cycleGANdata/vangogh2photo/vangogh2photo/trainB/2015-02-13 10:07:16.jpg  
  inflating: cycleGANdata/vangogh2photo/vangogh2photo/trainB/2015-02-13 13:57:11.jpg  
  inflating: cycleGANdata/vangogh2photo/vangogh2photo/trainB/2015-02-13 16:02:50.jpg  
  inflating: cycleGANdata/vangogh2photo/vangogh2photo/trainB/2015-02-13 16:45:49.jpg  
  inflating: cycleGANdata/vangogh2photo/vangogh2photo/trainB/2015-02-14 00:09:27.jpg  
  inflating: cyc

In [8]:
import os
import numpy as np
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from PIL import Image
from tqdm import tqdm

import torch
from torchvision import transforms
from torchvision.utils import save_image
from torchvision import transforms
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset

## Config


In [9]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# TRAIN_DIR = "/kaggle/input/cyclegan/horse2zebra/horse2zebra"
# VAL_DIR = "/kaggle/input/cyclegan/horse2zebra/horse2zebra"
TRAIN_DIR = "cycleGANdata/monet2photo/monet2photo"
VAL_DIR = "cycleGANdata/monet2photo/monet2photo"
BATCH_SIZE = 1
LEARNING_RATE = 1e-4
LAMBDA_IDENTITY = 0.0
LAMBDA_CYCLE = 10
NUM_WORKERS = 4
NUM_EPOCHS = 12
LOAD_MODEL = True
SAVE_MODEL = True
CHECKPOINT_GEN_H = "checkpoints/genh.pth.tar"
CHECKPOINT_GEN_Z = "checkpoints/genz.pth.tar"
CHECKPOINT_CRITIC_H = "checkpoints/critich.pth.tar"
CHECKPOINT_CRITIC_Z = "checkpoints/criticz.pth.tar"

transforms = A.Compose(
    [
        A.Resize(width=256, height=256),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255),
        ToTensorV2(),
    ],
    additional_targets={"image0": "image"},
)

## Utils


In [10]:
def save_checkpoint(model, optimizer, filename="/checkpoints/my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


## Dataset

In [11]:
class HorseZebraDataset(Dataset):
    def __init__(self, root_zebra, root_horse, transform=None):
        self.root_zebra = root_zebra
        self.root_horse = root_horse
        self.transform = transform

        self.zebra_images = os.listdir(root_zebra)
        self.horse_images = os.listdir(root_horse)
        self.length_dataset = max(len(self.zebra_images), len(self.horse_images)) # 1000, 1500
        self.zebra_len = len(self.zebra_images)
        self.horse_len = len(self.horse_images)

    def __len__(self):
        return self.length_dataset

    def __getitem__(self, index):
        zebra_img = self.zebra_images[index % self.zebra_len]
        horse_img = self.horse_images[index % self.horse_len]

        zebra_path = os.path.join(self.root_zebra, zebra_img)
        horse_path = os.path.join(self.root_horse, horse_img)

        zebra_img = np.array(Image.open(zebra_path).convert("RGB"))
        horse_img = np.array(Image.open(horse_path).convert("RGB"))

        if self.transform:
            augmentations = self.transform(image=zebra_img, image0=horse_img)
            zebra_img = augmentations["image"]
            horse_img = augmentations["image0"]

        return zebra_img, horse_img






## Disc_Model

In [12]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                4,
                stride,
                1,
                bias=True,
                padding_mode="reflect",
            ),
            nn.InstanceNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace=True),
        )

    def forward(self, x):
        return self.conv(x)


class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                in_channels,
                features[0],
                kernel_size=4,
                stride=2,
                padding=1,
                padding_mode="reflect",
            ),
            nn.LeakyReLU(0.2, inplace=True),
        )

        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(
                Block(in_channels, feature, stride=1 if feature == features[-1] else 2)
            )
            in_channels = feature
        layers.append(
            nn.Conv2d(
                in_channels,
                1,
                kernel_size=4,
                stride=1,
                padding=1,
                padding_mode="reflect",
            )
        )
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.initial(x)
        return torch.sigmoid(self.model(x))


def test():
    x = torch.randn((5, 3, 256, 256))
    model = Discriminator(in_channels=3)
    preds = model(x)
    print(preds.shape)


if __name__ == "__main__":
    test()

torch.Size([5, 1, 30, 30])


## Gen_Model

In [13]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, use_act=True, **kwargs):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, padding_mode="reflect", **kwargs)
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, **kwargs),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True) if use_act else nn.Identity(),
        )

    def forward(self, x):
        return self.conv(x)


class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            ConvBlock(channels, channels, kernel_size=3, padding=1),
            ConvBlock(channels, channels, use_act=False, kernel_size=3, padding=1),
        )

    def forward(self, x):
        return x + self.block(x)


class Generator(nn.Module):
    def __init__(self, img_channels, num_features=64, num_residuals=9):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                img_channels,
                num_features,
                kernel_size=7,
                stride=1,
                padding=3,
                padding_mode="reflect",
            ),
            nn.InstanceNorm2d(num_features),
            nn.ReLU(inplace=True),
        )
        self.down_blocks = nn.ModuleList(
            [
                ConvBlock(
                    num_features, num_features * 2, kernel_size=3, stride=2, padding=1
                ),
                ConvBlock(
                    num_features * 2,
                    num_features * 4,
                    kernel_size=3,
                    stride=2,
                    padding=1,
                ),
            ]
        )
        self.res_blocks = nn.Sequential(
            *[ResidualBlock(num_features * 4) for _ in range(num_residuals)]
        )
        self.up_blocks = nn.ModuleList(
            [
                ConvBlock(
                    num_features * 4,
                    num_features * 2,
                    down=False,
                    kernel_size=3,
                    stride=2,
                    padding=1,
                    output_padding=1,
                ),
                ConvBlock(
                    num_features * 2,
                    num_features * 1,
                    down=False,
                    kernel_size=3,
                    stride=2,
                    padding=1,
                    output_padding=1,
                ),
            ]
        )

        self.last = nn.Conv2d(
            num_features * 1,
            img_channels,
            kernel_size=7,
            stride=1,
            padding=3,
            padding_mode="reflect",
        )

    def forward(self, x):
        x = self.initial(x)
        for layer in self.down_blocks:
            x = layer(x)
        x = self.res_blocks(x)
        for layer in self.up_blocks:
            x = layer(x)
        return torch.tanh(self.last(x))


def test():
    img_channels = 3
    img_size = 256
    x = torch.randn((2, img_channels, img_size, img_size))
    gen = Generator(img_channels, 9)
    print(gen(x).shape)


if __name__ == "__main__":
    test()

torch.Size([2, 3, 256, 256])


## Train

In [14]:
def train_fn(
    disc_H, disc_Z, gen_Z, gen_H, loader, opt_disc, opt_gen, l1, mse):
    H_reals = 0
    H_fakes = 0
    loop = tqdm(loader, leave=True)

    for idx, (zebra, horse) in enumerate(loop):
        zebra = zebra.to(DEVICE)
        horse = horse.to(DEVICE)

        # Train Discriminators H and Z
        with torch.cuda.amp.autocast():
            fake_horse = gen_H(zebra)
            D_H_real = disc_H(horse)
            D_H_fake = disc_H(fake_horse.detach())
            H_reals += D_H_real.mean().item()
            H_fakes += D_H_fake.mean().item()
            D_H_real_loss = mse(D_H_real, torch.ones_like(D_H_real))
            D_H_fake_loss = mse(D_H_fake, torch.zeros_like(D_H_fake))
            D_H_loss = D_H_real_loss + D_H_fake_loss

            fake_zebra = gen_Z(horse)
            D_Z_real = disc_Z(zebra)
            D_Z_fake = disc_Z(fake_zebra.detach())
            D_Z_real_loss = mse(D_Z_real, torch.ones_like(D_Z_real))
            D_Z_fake_loss = mse(D_Z_fake, torch.zeros_like(D_Z_fake))
            D_Z_loss = D_Z_real_loss + D_Z_fake_loss

            # put it togethor
            D_loss = (D_H_loss + D_Z_loss) / 2

        opt_disc.zero_grad()
        D_loss.backward()
        opt_disc.step


        # Train Generators H and Z
        with torch.cuda.amp.autocast():
            # adversarial loss for both generators
            D_H_fake = disc_H(fake_horse)
            D_Z_fake = disc_Z(fake_zebra)
            loss_G_H = mse(D_H_fake, torch.ones_like(D_H_fake))
            loss_G_Z = mse(D_Z_fake, torch.ones_like(D_Z_fake))

            # cycle loss
            cycle_zebra = gen_Z(fake_horse)
            cycle_horse = gen_H(fake_zebra)
            cycle_zebra_loss = l1(zebra, cycle_zebra)
            cycle_horse_loss = l1(horse, cycle_horse)

            # identity loss (remove these for efficiency if you set lambda_identity=0)
            identity_zebra = gen_Z(zebra)
            identity_horse = gen_H(horse)
            identity_zebra_loss = l1(zebra, identity_zebra)
            identity_horse_loss = l1(horse, identity_horse)

            # add all togethor
            G_loss = (
                loss_G_Z
                + loss_G_H
                + cycle_zebra_loss * LAMBDA_CYCLE
                + cycle_horse_loss * LAMBDA_CYCLE
                + identity_horse_loss * LAMBDA_IDENTITY
                + identity_zebra_loss * LAMBDA_IDENTITY
            )

        opt_gen.zero_grad()
        G_loss.backward()
        opt_gen.step()
#         save_image(fake_horse * 0.5 + 0.5, f"/kaggle/working/results/photo_{idx}.png")
#         save_image(fake_zebra * 0.5 + 0.5, f"/kaggle/working/results/monet_{idx}.png")
        if idx % 20 == 0:
            save_image(fake_horse * 0.5 + 0.5, f"results/photo_{idx}.png")
            save_image(fake_zebra * 0.5 + 0.5, f"results/monet_{idx}.png")

        loop.set_postfix(H_real=H_reals / (idx + 1), H_fake=H_fakes / (idx + 1))




In [17]:
import time
disc_H = Discriminator(in_channels=3).to(DEVICE)
disc_Z = Discriminator(in_channels=3).to(DEVICE)
gen_Z = Generator(img_channels=3, num_residuals=9).to(DEVICE)
gen_H = Generator(img_channels=3, num_residuals=9).to(DEVICE)
opt_disc = Adam(
    list(disc_H.parameters()) + list(disc_Z.parameters()),
    lr=LEARNING_RATE,
    betas=(0.5, 0.999),
)

opt_gen = Adam(
    list(gen_Z.parameters()) + list(gen_H.parameters()),
    lr=LEARNING_RATE,
    betas=(0.5, 0.999),
)

L1 = nn.L1Loss()
mse = nn.MSELoss()

if LOAD_MODEL:
    load_checkpoint(
        CHECKPOINT_GEN_H,
        gen_H,
        opt_gen,
        LEARNING_RATE,
    )
    load_checkpoint(
        CHECKPOINT_GEN_Z,
        gen_Z,
        opt_gen,
        LEARNING_RATE,
    )
    load_checkpoint(
        CHECKPOINT_CRITIC_H,
        disc_H,
        opt_disc,
        LEARNING_RATE,
    )
    load_checkpoint(
        CHECKPOINT_CRITIC_Z,
        disc_Z,
        opt_disc,
        LEARNING_RATE,
    )

dataset = HorseZebraDataset(
    root_horse=TRAIN_DIR + "/testA",
    root_zebra=TRAIN_DIR + "/testB",
    transform=transforms,
)
val_dataset = HorseZebraDataset(
    root_horse=VAL_DIR + "/testA",
    root_zebra=VAL_DIR + "/testB",
    transform=transforms,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=False,
    pin_memory=True,
)
loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)
startTime = time.time()
print("===== start time {} =====".format(startTime))

for epoch in range(NUM_EPOCHS):
    print('start epoch {} with time {} -----------'.format(epoch, time.time()))
    train_fn(
        disc_H,
        disc_Z,
        gen_Z,
        gen_H,
        loader,
        opt_disc,
        opt_gen,
        L1,
        mse,
        )

    if SAVE_MODEL:
        save_checkpoint(gen_H, opt_gen, filename=CHECKPOINT_GEN_H)
        save_checkpoint(gen_Z, opt_gen, filename=CHECKPOINT_GEN_Z)
        save_checkpoint(disc_H, opt_disc, filename=CHECKPOINT_CRITIC_H)
        save_checkpoint(disc_Z, opt_disc, filename=CHECKPOINT_CRITIC_Z)
print('===== during time {} ====='.format(time.time()-startTime))



=> Loading checkpoint
=> Loading checkpoint


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


=> Loading checkpoint
=> Loading checkpoint
===== start time 1701463001.8920326 =====
start epoch 0 with time 1701463001.8922503 -----------


100%|██████████| 751/751 [03:20<00:00,  3.74it/s, H_fake=0.508, H_real=0.504]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
start epoch 1 with time 1701463204.4885652 -----------


...

=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
start epoch 10 with time 1701464944.3910954 -----------


100%|██████████| 751/751 [03:10<00:00,  3.95it/s, H_fake=0.508, H_real=0.504]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
start epoch 11 with time 1701465136.4927902 -----------


100%|██████████| 751/751 [03:09<00:00,  3.96it/s, H_fake=0.508, H_real=0.504]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
===== during time 2325.9190039634705 =====
